In [1]:
import pandas as pd
import random
import sys

from sklearn.impute import SimpleImputer
from num2words import num2words
from scipy.stats import truncnorm

sys.path.append('../')
from src.transformers_ import HeightWeightHandler, AgeExtractor
from src.utils import *

pd.set_option('display.max_rows', 500)
df = pd.DataFrame()

In [2]:
df['age'] = generate_age_column()
df['height_weight'] = generate_height_weight_column()
df['transportation'] = generate_categorical_column(['Automobile', 'Public_Transportation', 'Motorbike', 'Walking', 'Bike'])
df['recorded'] = generate_yes_no_column()
df['price'] = generate_missing_numeric_column()
df['walked'] = generate_missing_numeric_column(1,10)
df['eaten'] = generate_missing_numeric_column(1,3)

In [3]:
df.head(10)

,age,height_weight,transportation,recorded,price,walked,eaten
0,2,"1.5638571403564765,one hundred and ten",Bike,no,NaN,8.0,NaN
1,67,1.5118764321530023_fifty-four,Walking,yes,2036.0,8.0,2.0
2,twenty-three,1.6347239981610846;eighty-two,Bike,no,NaN,9.0,NaN
3,75,1.85880599561046-one hundred and nine,Automobile,yes,NaN,NaN,3.0
4,75,2.215629077974412_one hundred and thirty,Bike,yes,2564.0,NaN,NaN
5,76,1.8696445382175086_forty-eight,Public_Transportation,no,3407.0,NaN,2.0
6,ninety-two,1.821427472580821;64,Motorbike,no,6229.0,8.0,2.0
7,2,"1.8165169673762127,sixty-nine",Walking,yes,6308.0,NaN,3.0
8,eight,1.591921330518903;eighty-one,Automobile,yes,7381.0,NaN,2.0
9,thirty-four,1.9742732062966672_108,Bike,no,NaN,NaN,NaN


In [4]:
df.isnull().sum()

age                0
height_weight      0
transportation     0
recorded           0
price             50
walked            57
eaten             53
dtype: int64

In [5]:
mean_imputer = SimpleImputer(strategy='mean')
mean_col = ['price', 'walked']
df[mean_col] = mean_imputer.fit_transform(df[mean_col])

const_imputer = SimpleImputer(strategy='constant', fill_value=0)
const_col = ['eaten']
df[const_col] = const_imputer.fit_transform(df[const_col])

In [5]:
age_extracted = AgeExtractor().fit_transform(df['age'])
heights_weights = HeightWeightHandler('height_weight').fit_transform(df)
transport_dummy = pd.get_dummies(df['transportation'])
df['recorded'] = df['recorded'].replace({'no': 0, 'yes': 1})

In [6]:
df.drop(['age', 'height_weight', 'transportation'], axis=1, inplace=True)

In [7]:
df = pd.concat([age_extracted, heights_weights, transport_dummy, df], axis=1)

In [8]:
df.head(10)

,age,height,weight,Automobile,Bike,Motorbike,Public_Transportation,Walking,recorded,price,walked,eaten
0,82,1.471906,103.0,False,False,False,False,True,1,8451.0,6.26087,2.0
1,94,1.505534,90.0,False,True,False,False,False,1,5659.0,6.26087,0.0
2,20,1.611056,85.0,False,False,False,False,True,1,2747.0,9.00000,2.0
3,64,1.546454,80.0,False,False,False,True,False,0,5659.0,6.26087,0.0
4,29,1.458686,109.0,False,True,False,False,False,1,3851.0,6.26087,2.0
5,15,1.433727,42.0,False,False,True,False,False,0,9223.0,4.00000,2.0
6,32,1.340496,131.0,True,False,False,False,False,1,4956.0,6.26087,0.0
7,73,1.338687,128.0,False,True,False,False,False,0,5659.0,4.00000,0.0
8,100,2.014705,104.0,False,True,False,False,False,0,5659.0,6.26087,0.0
9,73,1.922467,50.0,False,False,False,True,False,1,5659.0,3.00000,0.0
